# The `GeneNeighborhood` API

## ...And a First Cut at `DuckDB` Database

*Mark Green - 3/11/2023*

This notebook outlines the `GeneNeighborhood` class. With the DIOPT-provided Gene Homology and identification data stored in the `DuckDB`, this class calls the database to get the neighborhood of immediately connected nodes (genes) to the target geneID. The class builds a `networkx` graph object, and also a `numpy` weighted adjacency matrix. 

An example call is given, as well as the code for initializing the `DuckDB` - which can be done either in memory or on disk.

The `pandas` equivalent is also listed below, for easy reading and experimentation. I don't think we should use this, as I don't believe it is as efficient with data types, nor as flexible future scaling when compared to the `GeneNeighborhood` class. 

A future extension for this will be to essentially append successive neighborhood graphs. IE, an initial see gene is selected and a neighborhood is built. Then successive genes are "clicked" by the user and those gene neighborhoods are fetched and appended to the existing visualized network. Successively expanding the outward, incorporating a new gene homology neighborhood with each "click". 

Additionally, this framework should serve to allow for easy appending of additional node or edge attributes by modifying the `.Graph()` method. For instance, if we want to try analyzing for edge communities. 

### `../data/` Setup

Example setup for the folders, for this script to work. 

```markdown
docs
└── duck.db
data
├── Gene_Information.tsv
└── Ortholog_Pair_Best.tsv
```

In [1]:
class GeneNeighborhood:
    """
    Generates a local gene homology neighborhood for a given gene
    Inputs:
        ----::---------------------------------------------------
        gid :: int 
            :: NCBI gene id number
        ----::---------------------------------------------------
        con :: SQL (DuckDB) db connection
            :: 2 tables created from .tsv files provided by DIOPT:
            ::    + tblGeneInfo <- 'Gene_Information.tsv'
            ::    + tblOrthologPairs <- Ortholog_Pair_Best.tsv
        ----::---------------------------------------------------
    Dependencies: 
       + duckdb
       + networkx
       + numpy
       + scipy.sparse
    """
    
    def __init__(self, gid, con):
        self.gid = gid
        self.con = con
        self.__CallSQL()
        self.__AdjacencyList()
    
    # API calling SQL db to get data for gene homology neighborhood graph. 
    def CallSQL(self):
        
        # drop & build a temp table to list the neighborhood of targetGenes 
        self.con.execute(
            """
            DROP TABLE IF EXISTS tempTargetGenes; 
            CREATE TEMP TABLE tempTargetGenes AS 
                SELECT DISTINCT geneid1 AS geneid FROM tblOrthologPairs 
                WHERE geneid1 = ? OR geneid2 = ?
            """
            ,[self.gid,self.gid])
        
        # assign node attribute components
        self.targetGenes = self.con.execute(
            """
            SELECT geneid FROM tempTargetGenes ORDER BY geneid
            """
            ).fetchall()
        
        self.geneInfoNames = self.con.execute(
            """
            SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE TABLE_NAME = N'tblGeneInfo'
            """
            ).fetchall()
            
        self.nodeList = self.con.execute(
            """
            SELECT * FROM tblGeneInfo WHERE geneid IN (
                SELECT geneid FROM tempTargetGenes ORDER BY geneid)
            """
            ).fetchall()
        
        # assign edge attribute components
        self.edgeWeights = self.con.execute(
            """
            SELECT geneid1, geneid2, weight FROM tblOrthologPairs 
            WHERE geneid1 IN (SELECT geneid FROM tempTargetGenes) 
                AND geneid2 IN (SELECT geneid FROM tempTargetGenes)
            """
            ).fetchall()
        
        self.orthologPairNames = self.con.execute(
            """
            SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE TABLE_NAME = N'tblOrthologPairs'
            """
            ).fetchall()
        
        self.edgeList = self.con.execute(
            """
            SELECT * FROM tblOrthologPairs 
            WHERE geneid1 IN (SELECT geneid FROM tempTargetGenes) 
                AND geneid2 IN (SELECT geneid FROM tempTargetGenes)
            """
            ).fetchall()
    
    __CallSQL = CallSQL
    
    # Formats the node and edge attributes to create a NetworkX Graph() object.     
    def Graph(self, write=False):

        node_attr = dict(zip(
            [i for i in range(len(self.targetGenes))],
            [dict(zip(
                [i for (i,) in self.geneInfoNames],
                [i for i in n]
            )) for n in self.nodeList]
        ))        

        edge_attr = np.array([dict(zip(
            [i for (i,) in self.orthologPairNames],
            [i for i in e]
        )) for e in self.edgeList])

        graph_edgelist = [tuple(i) for i in np.column_stack((self.adjList[:,:2],edge_attr))]

        G = nx.from_edgelist(graph_edgelist)
        nx.set_node_attributes(G, node_attr)
        
        if write:
            gexf_path = '../graphs/'+str(self.gid)+'.gexf'
            nx.write_gexf(G, gexf_path)
            print(f"{gexf_path} written.")
        
        return G
    
    __Graph = Graph
    
    # creates weighted edgelist with reindexed node ids.
    def AdjacencyList(self):
        
        node_index_map = dict(zip(
            [i for (i,) in self.targetGenes],
            [i for i in range(len(self.targetGenes))]
        ))
        adj_list = np.array(self.edgeWeights).astype(object)
        adj_list[:,:2] = np.vectorize(node_index_map.get)(adj_list[:,:2])
        self.adjList = adj_list
        
    __AdjacencyList = AdjacencyList
    
    # creates weighted adjacency matrix.
    def AdjacencyMatrix(self):  
        
        i, j, weight = self.adjList[:,0], self.adjList[:,1], self.adjList[:,2]
        dim = max(len(set(i)), len(set(j)))
        adj_mat = sps.lil_matrix((dim, dim))
        for i,j,w in zip(i,j,weight):
            adj_mat[i,j] = w

        return adj_mat.todense()


In [4]:
import duckdb
import networkx as nx
import numpy as np
import scipy.sparse as sps

# Initialize db connection
con = duckdb.connect(database='duck.db', read_only=True)

# ### Alternatively this code could be run to build the dabase in memory from the ./tsv files ###

# con = duckdb.connect(database=':memory:')

### build db from .tsv sourcefiles
# con.execute(
#     """
#     DROP TABLE IF EXISTS tblGeneInfo;
#     CREATE TABLE tblGeneInfo AS 
#         SELECT * FROM read_csv(
#             '../data/Gene_Information.tsv', delim='\t', 
#             header=True, AUTO_DETECT=TRUE
#         );
#     """
# )
# ### NOTE: this script should be run in a terminal, may crash Jupyter...
# con.execute(
#     """
#     DROP TABLE IF EXISTS tblOrthologPairs;
#     CREATE TABLE tblOrthologPairs AS 
#         SELECT * FROM read_csv(
#             '../data/Ortholog_Pair_Best.tsv', delim='\t', 
#             header=True, AUTO_DETECT=TRUE
#         );
#     ALTER TABLE tblOrthologPairs
#     RENAME COLUMN score TO weight;
#     """
# )

### end alternate ###

# Set input ID, visualize graph and write out gexf file for vizualization
gid=2911

GN = GeneNeighborhood(gid=gid, con=con)

print(GN.AdjacencyMatrix())
    
G = GN.Graph(write=True)

# nx.draw(G)

[[0. 1. 1. ... 1. 1. 1.]
 [1. 0. 4. ... 2. 1. 8.]
 [1. 4. 0. ... 1. 3. 1.]
 ...
 [1. 2. 1. ... 0. 2. 3.]
 [1. 1. 3. ... 2. 0. 2.]
 [1. 8. 1. ... 3. 2. 0.]]
../graphs/2911.gexf written.


In [ ]:
import networkx as nx
import numpy as np
import pandas as pd

# Here is the pandas version of the above API

ortholog_pairs = pd.read_csv('../data/Ortholog_Pair_Best.tsv', sep='\t')
gene_info = pd.read_csv('../data/Gene_Information.tsv', sep='\t')
    
# set target geneID
gidin = 2911

# get ortholog pairs of target gene
gidin_orthologs = ortholog_pairs[
    (ortholog_pairs['geneid1']==gidin) | 
    (ortholog_pairs['geneid2']==gidin)
]

# get ortholog pairs within target gene's ortholog pairs' nodes
gidin_all = ortholog_pairs[
    (ortholog_pairs['geneid1'].isin(gidin_orthologs['geneid1'])) &
    (ortholog_pairs['geneid2'].isin(gidin_orthologs['geneid2']))
]

# arrange adjacency matrix
gidin_adj = pd.crosstab(
    gidin_all['geneid1'], gidin_all['geneid2'],
    values=gidin_all['score'], aggfunc='sum'
)
gidin_nodes = gidin_adj.columns.values
gidin_mat = gidin_adj.to_numpy()

# get node attributes dict
gidin_nodes_attr = gene_info[gene_info['geneid'].isin(gidin_nodes)]\
                                                .reset_index(drop=True)\
                                                .to_dict('index')

G2 = nx.from_numpy_array(gidin_mat)
nx.set_node_attributes(G2, gidin_nodes_attr)

# nx.write_gexf(G2, 'GeneNeighborhood2.gexf')

/tmp/ipykernel_26212/3933273306.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_info = pd.read_csv('../data/Gene_Information.tsv', sep='\t')


In [6]:
nx.get_node_attributes(G2)

TypeError: get_node_attributes() missing 1 required positional argument: 'name'

A consideration to further increase performance will be to cache the API call results (graphs) in a `redis` or similar database, for users who are clicking through many gene IDs (this will limit I/O pressure from SQL db calls.